### Hartree-Fock Solutions of Small Atomic Systems (Central-Field Approximation)


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact, interactive, Dropdown, HBox, Layout, widgets
import pandas as pd
from IPython.core.display import HTML
#import matplotlib
#matplotlib.use('TkAgg')


In [10]:

#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
#      SUBROUTINE INIT
# initializes constants
# initializes arrays for input parameters
#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC


def INIT():
    global ANGMOM, FACTRL, MAXSTP, MAXSTT, NSTATE, E, FOCK, RHO, PSTOR, PHI, PSIIN, PSIOUT, NOCC, ID, FACTRL, HBARM, CHARGE, \
           ABOHR, IKEN, ICEN, IVEN, IVEE, IVEX, IKTOT, IVTOT, ITOT, ITOT_LAST 
    MAXSTP = 100000                               #maximum size of data arrays
    MAXSTT = 16                                  # maximum number of states allowed
    NSTATE = MAXSTT                             #total number of states in the calculation
    E = np.zeros((MAXSTT+1, 9))                 # (ROW=MAXSTT+1, COLUMN=8)/all energies of all states
    FOCK = np.zeros((MAXSTP+1, MAXSTT))         #Fock terms
    RHO = np.zeros(MAXSTP+1)                    #density
    PSTOR = np.zeros((MAXSTP+1, MAXSTT))        #radial wave functio
    PHI = np.zeros(MAXSTP+1)                    #electron potential
    PSIIN = np.zeros(MAXSTP+1)                  #homogeneous solutions
    PSIOUT = np.zeros(MAXSTP+1)                 #homogeneous solutions
    NOCC = np.zeros(MAXSTT+1)                   #occupation of each state
    ANGMOM = np.zeros(MAXSTT)                 #angular momentum of each state
    #ID = ["1S", "2S", "2P","3S","3P","4S","3D","4P", "5S", "4D", "5P", "6S", "5D", "4F", "6P", "Total"]                    #state identification
    FACTRL = np.zeros(21)                       #array of factorials
    HBARM = 7.6359                              # hbar^2/m for the electron (eV-A**2)
    CHARGE = 14.409                             # square of electron charge (eV-A)
    ABOHR = HBARM / CHARGE                      # Bohr radius (A)
    IKEN = 0                                    #index energy types
    ICEN = 1                                    #index energy types
    IVEN = 2                                    #index energy types
    IVEE = 3                                    #index energy types
    IVEX = 4                                    #index energy types
    IKTOT = 5                                   #index energy totals
    IVTOT = 6                                   #index energy totals
    ITOT = 7                                    #index energy totals
    ITOT_LAST = 8                               #index previous step energy totals
    ANGMOM[0] = 0
    ANGMOM[1] = 0
    ANGMOM[2] = 1
    ANGMOM[3] = 0
    ANGMOM[4] = 1
    ANGMOM[5] = 0
    ANGMOM[6] = 2
    ANGMOM[7] = 1
    ANGMOM[8] = 0
    ANGMOM[9] = 2
    ANGMOM[10] = 1
    ANGMOM[11] = 0
    ANGMOM[12] = 2
    ANGMOM[13] = 3
    ANGMOM[14] = 1
    
    ID = ["1S", "2S", "2P", "3S", "3P", "4S","3D","4P", "5S", "4D", "5P", "6S", "5D", "4F", "6P", "Total"]
      
    FACTRL[0] = 1              #factorials
    for I in range(1, 21):
        FACTRL[I] = I*FACTRL[I-1]
     
    

#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
#      SUBROUTINE PARAM
# gets parameters from screen     
# maps menu variables to program variables
# calculates all derivative parameters
#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
    
def PARAM(znum, nocc1s, nocc2s, nocc2p, nocc3s, nocc3p, nocc4s, nocc3d, nocc4p, nocc5s, nocc4d, nocc5p, nocc6s, nocc5d, nocc4f, nocc6p):
    global Z, NR, NOCC, ZCHARG, NE, DR, RMAX 
    
    
    Z = znum #int(input("Enter nuclear charge (>=1, <=20), Z: "))    # get the nuclear charge/number of protons in nucleus
      
    NOCC[0] = nocc1s #int(input("Enter number of electrons in the 1s state (>=0, <=2): ")) #Occupation of 1s state
    NOCC[1] = nocc2s #int(input("Enter number of electrons in the 2s state (>=0, <=2): ")) #Occupation of 2s state
    NOCC[2] = nocc2p #int(input("Enter number of electrons in the 2p state (>=0, <=6): ")) #Occupation of 2p state
    NOCC[3] = nocc3s #int(input("Enter number of electrons in the 3s state (>=0, <=2): ")) #Occupation of 3s state
    NOCC[4] = nocc3p #int(input("Enter number of electrons in the 3p state (>=0, <=6): ")) #Occupation of 3p state
    NOCC[5] = nocc4s #int(input("Enter number of electrons in the 3d state (>=0, <=2): ")) #Occupation of 4s state
    NOCC[6] = nocc3d #int(input("Enter number of electrons in the 4s state (>=0, <=10): ")) #Occupation of 3d state
    NOCC[7] = nocc4p #int(input("Enter number of electrons in the 4p state (>=0, <=6): ")) #Occupation of 4p state
    NOCC[8] = nocc5s #int(input("Enter number of electrons in the 5s state (>=0, <=2): ")) #Occupation of 5s state
    NOCC[9] = nocc4d #int(input("Enter number of electrons in the 4d state (>=0, <=10): ")) #Occupation of 4d state
    NOCC[10] = nocc5p #int(input("Enter number of electrons in the 5p state (>=0, <=6): ")) #Occupation of 5p state
    NOCC[11] = nocc6s #int(input("Enter number of electrons in the 6s state (>=0, <=2): ")) #Occupation of 6s state
    NOCC[12] = nocc5d #int(input("Enter number of electrons in the 5d state (>=0, <=10): ")) #Occupation of 5d state
    NOCC[13] = nocc4f #int(input("Enter number of electrons in the 4f state (>=0, <=14): ")) #Occupation of 4f state
    NOCC[14] = nocc6p #int(input("Enter number of electrons in the 6p state (>=0, <=6): ")) #Occupation of 6p state
    
    # Get the lattice parameters
    DR = 0.01 #float(input("Enter radial step size (in Angstroms) (>=0.0001, <=0.5): "))
    RMAX = 5 #float(input("Enter outer radius of the lattice (in Angstroms) (>=0.01, <=10): "))

     
    #calculate derivative parameters:
    ZCHARG =Z*CHARGE      #nuclear charge
    NR = int(RMAX/DR)      #number of radial steps
    
    NOCC[NSTATE]=0     #total number of electrons
    for I in range(NSTATE):
        NOCC[NSTATE] += NOCC[I]
        
    if NR >= MAXSTP:
        print("The resulting lattice has %d points" % NR)
        print("The maximum number of points allowed is %d" % MAXSTP)
        print("Try again")
        exit()
    
    NE = NOCC[NSTATE]  # provides the same result as above loop
    return Z, nocc1s, nocc2s, nocc2p, nocc3s, nocc3p, nocc3d, nocc4s, nocc4p, nocc5s, nocc4d, nocc5p, nocc6s, nocc5d, nocc4f, nocc6p 



#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
#      SUBROUTINE ARCHON
# find the Hartree-Fock wave functions for the specified atom
#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
    

def ARCHON():
    global MIX
        
#   begin iterations with a good guess
    MIX = 1                    #no old density to mix with new
    ZSTAR = Z  
    HYDRGN(ZSTAR,PSTOR)  # find hydrogenic wave functions
    ENERGY(E,FOCK,RHO,PHI,PSTOR)  # energy

#   optimal ZSTAR using Virial theorem
    ZSTAR = -Z * (E[NSTATE,IVTOT]) / (2 * E[NSTATE,IKTOT])        #effective nuclear charge #taking into account the shielding effects of other electrons.
    
    HYDRGN(ZSTAR,PSTOR)  #find new hydrogenic wave functions 
    ENERGY(E,FOCK,RHO,PHI,PSTOR) # and energies

    print("Nuclear Charge (Z): %d" % Z)
    print("Number of Electrons: %f" % NE)
    print("Occupation of the States: ")
    print("  1s: %d" % NOCC[0])
    print("  2s: %d" % NOCC[1])
    print("  2p: %d" % NOCC[2])
    print("  3s: %d" % NOCC[3])
    print("  3p: %d" % NOCC[4])
    print("  4s: %d" % NOCC[5])
    print("  3d: %d" % NOCC[6])
    print("  4p: %d" % NOCC[7])
    print("  5s: %d" % NOCC[8])
    print("  4d: %d" % NOCC[9])
    print("  5p: %d" % NOCC[10])
    print("  6s: %d" % NOCC[11])
    print("  5d: %d" % NOCC[12])
    print("  4f: %d" % NOCC[13])
    print("  6p: %d" % NOCC[14])    
    print("Radial Step (Angstroms): %f" % DR)
    print("Maximum Radius (Angstroms): %f" % (DR * NR))
    print("Effective Nuclear Charge (Z*): %f" % ZSTAR)
    print("Hydrogenic Orbital Gives Effective Nuclear Charge (Z*): %f" % ZSTAR)
    print("\n")
    print("All energies are in electron volts (eV).")



    MIX = 0.5                    #mix old and new density for stability
    ITER = 0                    #zero iteration counters
    ISTART = 0                                        
    ISTOP = 0
    NITER = 300




#   output initial energies
    print("*"*50)
    print("Itration = ", ITER)
    print("*"*50)
    print("\n")
    for IS in range(NSTATE):
        if NOCC[IS] != 0:
            print('{:<7s} {:<6s} {:<20s} {:<20s} {:<20s} {:<20s} {:<20s} {:<20s}'.format("state1", "Nocc", "Ktot", "Ven", "Vee", "Vex", "Vtot", "Etot"))
            print('{:<7s} {:<6.1f} {:<18.14f} {:<18.14f} {:<18.14f} {:<18.14f} {:<18.14f} {:<18.14f}'.format(ID[IS], NOCC[IS], E[IS,IKTOT], E[IS,IVEN], E[IS,IVEE], E[IS,IVEX], E[IS,IVTOT], E[IS,ITOT]))
  
    print("\n")
    print("*"*50)
    print("The total Energy convergence criterion is 1E-7 eV")
    print("\n")
    
    ISTART = ISTOP+1          #update iteration counters
    ISTOP = ISTOP+NITER
    for ITER in range(ISTART,ISTOP+1):
        
        for STATE in range(NSTATE):
            if NOCC[STATE] != 0:       #loop over all occpd states
                ESP = E[STATE,ITOT]        #single particle energy
                if ESP > 0:
                    ESP = -12   #keep particle bound

    #           find single part wave function
                SNGWFN(STATE,ESP,PSTOR,FOCK,PHI)


#       calculate new energies and output
        ENERGY(E,FOCK,RHO,PHI,PSTOR)
        print("Itration:", ITER,  "             ", "Total Energy:", E[NSTATE,ITOT], "eV")
    
  
        if abs(Total_Energy_Diff) <= 1E-7:  # Check for greater than or equal to 1E-5
            print("*"*50)
            print("Itration = ", ITER)
            #print("{0:^200}" .format("Itration = ", ITER))
            print("*"*50)
            print("\n")
            for IS in range(NSTATE):
                if NOCC[IS] != 0:
                    print('{:<7s} {:<6s} {:<20s} {:<20s} {:<20s} {:<20s} {:<20s} {:<20s}'.format("state", "Nocc", "Ktot", "Ven", "Vee", "Vex", "Vtot", "Etot"))
                    print('{:<7s} {:<6.1f} {:<18.14f} {:<18.14f} {:<18.14f} {:<18.14f} {:<18.14f} {:<18.14f}'.format(ID[IS], NOCC[IS], E[IS,IKTOT], E[IS,IVEN], E[IS,IVEE], E[IS,IVEX], E[IS,IVTOT], E[IS,ITOT]))
            print("\n")
            print("*"*50)
            print("\n")
            print("TOTALS")
            print("NE :", NE)
            print("KTOT :", E[NSTATE, IKTOT], "eV")
            print("VENTOT :", E[NSTATE, IVEN], "eV")
            print("VEETOT :", E[NSTATE, IVEE], "eV")
            print("VEXTOT :", E[NSTATE, IVEX], "eV")
            print("VENTOT + VEETOT + VEXTOT :", E[NSTATE, IVTOT], "eV")
            print("Total Energy :", E[NSTATE,ITOT], "eV")
            print("Previous Total Energy :", E[NSTATE,ITOT_LAST], "eV")
            print("Total_Energy_Diff", Total_Energy_Diff, "eV")
            print("Final Converged Total Energy is :", E[NSTATE,ITOT], "eV")
            
            fig = plt.figure(figsize=[30,10])
            X = np.linspace(0, NR, NR)
            R = X*DR
            '''
            #fig = plt.figure(figsize=(20, 10))
            # Part (a) and (b)
            ax1 = fig.add_subplot(1,2,1)
            ax1.plot(R, NOCC[0]*(PSTOR[:500, 0]**2), 'b', linewidth = 3.0, label = '1s orbital') # thia takes X axis as index values
            ax1.plot(R, NOCC[1]*(PSTOR[:500, 1]**2), 'r', linewidth = 3.0, label = '2s orbital')
            ax1.plot(R, NOCC[2]*(PSTOR[:500, 2]**2), 'g', linewidth = 3.0, label = '2p orbital')
            ax1.plot(R, NOCC[3]*(PSTOR[:500, 3]**2), 'c', linewidth = 3.0, label = '3s orbital')
            ax1.plot(R, NOCC[4]*(PSTOR[:500, 4]**2), 'y', linewidth = 3.0, label = '3p orbital')
            ax1.plot(R, NOCC[5]*(PSTOR[:500, 5]**2), 'm', linewidth = 3.0, label = '4s orbital')
            ax1.plot(R, NOCC[6]*(PSTOR[:500, 6]**2), 'k', linewidth = 3.0, label = '3d orbital')
            ax1.plot(R, NOCC[7]*(PSTOR[:500, 7]**2), 'b:', linewidth = 3.0, label = '4p orbital')
            ax1.set(ylabel='Probablilty Density')
            ax1.set(xlabel='R (Angstroms)')
            ax1.set_xlim([0,2])
            #ax1.set_ylim([0,40])
            plt.legend(loc=1)
            plt.grid(True)

            ax2 = fig.add_subplot(1,2,2)
            ax2.plot(R, RHO[:500], 'b', linewidth=3.0, label='Total density of all states')
            ax2.set(xlabel='R (Angstroms)')
            ax2.set(ylabel='Probablilty Density')
            #ax2.set_xlim([0,100])
            #ax2.set_ylim([0,0.02])
            plt.legend(loc=1)
            #plt.legend()
            plt.grid(True)
            '''
            ax1 = fig.add_subplot(4,4,1)
            ax1.plot(R, NOCC[0]*(PSTOR[:500, 0]**2), 'b', linewidth = 3.0, label = '1s orbital') # thia takes X axis as index values
            ax1.set(ylabel='Probablilty Density')
            ax1.set(xlabel='R (Angstroms)')
            ax1.set_xlim([0,4])
            #ax1.set_ylim([0,40])
            plt.legend(loc=1)
            plt.grid(True)
            ax2 = fig.add_subplot(4,4,2)
            ax2.plot(R, NOCC[1]*(PSTOR[:500, 1]**2), 'r', linewidth = 3.0, label = '2s orbital') # thia takes X axis as index values
            ax2.set(ylabel='Probablilty Density')
            ax2.set(xlabel='R (Angstroms)')
            ax2.set_xlim([0,4])
            #ax2.set_ylim([0,40])
            plt.legend(loc=1)
            plt.grid(True) 
            ax3 = fig.add_subplot(4,4,3)
            ax3.plot(R, NOCC[2]*(PSTOR[:500, 2]**2), 'g', linewidth = 3.0, label = '2p orbital') # thia takes X axis as index values
            ax3.set(ylabel='Probablilty Density')
            ax3.set(xlabel='R (Angstroms)')
            ax3.set_xlim([0,4])
            #ax3.set_ylim([0,40]) 
            plt.legend(loc=1)
            plt.grid(True)
            ax4 = fig.add_subplot(4,4,4)
            ax4.plot(R, NOCC[3]*(PSTOR[:500, 3]**2), 'c', linewidth = 3.0, label = '3s orbital') # thia takes X axis as index values
            ax4.set(ylabel='Probablilty Density')
            ax4.set(xlabel='R (Angstroms)')
            ax4.set_xlim([0,4])
            #ax4.set_ylim([0,40])
            plt.legend(loc=1)
            plt.grid(True)
            ax5 = fig.add_subplot(4,4,5)
            ax5.plot(R, NOCC[4]*(PSTOR[:500, 4]**2), 'y', linewidth = 3.0, label = '3p orbital') # thia takes X axis as index values
            ax5.set(ylabel='Probablilty Density')
            ax5.set(xlabel='R (Angstroms)')
            ax5.set_xlim([0,4])
            #ax5.set_ylim([0,40])
            plt.legend(loc=1)
            plt.grid(True)
            ax6 = fig.add_subplot(4,4,6)
            ax6.plot(R, NOCC[5]*(PSTOR[:500, 5]**2), 'm', linewidth = 3.0, label = '4s orbital') # thia takes X axis as index values
            ax6.set(ylabel='Probablilty Density')
            ax6.set(xlabel='R (Angstroms)')
            ax6.set_xlim([0,4])
            #ax6.set_ylim([0,40]) 
            plt.legend(loc=1)
            plt.grid(True)
            ax7 = fig.add_subplot(4,4,7)
            ax7.plot(R, NOCC[6]*(PSTOR[:500, 6]**2), 'k', linewidth = 3.0, label = '3d orbital') # thia takes X axis as index values
            ax7.set(ylabel='Probablilty Density')
            ax7.set(xlabel='R (Angstroms)')
            ax7.set_xlim([0,4])
            #ax7.set_ylim([0,40]) 
            plt.legend(loc=1)
            plt.grid(True)
            ax8 = fig.add_subplot(4,4,8)
            ax8.plot(R, NOCC[7]*(PSTOR[:500, 7]**2), 'b', linewidth = 3.0, label = '4p orbital') # thia takes X axis as index values
            ax8.set(ylabel='Probablilty Density')
            ax8.set(xlabel='R (Angstroms)')
            ax8.set_xlim([0,4])
            #ax8.set_ylim([0,40])
            plt.legend(loc=1)
            plt.grid(True)
            ax9 = fig.add_subplot(4,4,9)
            ax9.plot(R, NOCC[8]*(PSTOR[:500, 8]**2), 'r', linewidth = 3.0, label = '5s orbital') # thia takes X axis as index values
            ax9.set(ylabel='Probablilty Density')
            ax9.set(xlabel='R (Angstroms)')
            ax9.set_xlim([0,4])
            #ax9.set_ylim([0,40])
            plt.legend(loc=1)
            plt.grid(True) 
            ax10 = fig.add_subplot(4,4,10)
            ax10.plot(R, NOCC[9]*(PSTOR[:500, 9]**2), 'g', linewidth = 3.0, label = '4d orbital') # thia takes X axis as index values
            ax10.set(ylabel='Probablilty Density')
            ax10.set(xlabel='R (Angstroms)')
            ax10.set_xlim([0,4])
            #ax10.set_ylim([0,40]) 
            plt.legend(loc=1)
            plt.grid(True)
            ax11 = fig.add_subplot(4,4,11)
            ax11.plot(R, NOCC[10]*(PSTOR[:500, 10]**2), 'c', linewidth = 3.0, label = '5p orbital') # thia takes X axis as index values
            ax11.set(ylabel='Probablilty Density')
            ax11.set(xlabel='R (Angstroms)')
            ax11.set_xlim([0,4])
            #ax11.set_ylim([0,40])
            plt.legend(loc=1)
            plt.grid(True)
            ax12 = fig.add_subplot(4,4,12)
            ax12.plot(R, NOCC[11]*(PSTOR[:500, 11]**2), 'y', linewidth = 3.0, label = '6s orbital') # thia takes X axis as index values
            ax12.set(ylabel='Probablilty Density')
            ax12.set(xlabel='R (Angstroms)')
            ax12.set_xlim([0,4])
            #ax12.set_ylim([0,40])
            plt.legend(loc=1)
            plt.grid(True)
            ax13 = fig.add_subplot(4,4,13)
            ax13.plot(R, NOCC[12]*(PSTOR[:500, 12]**2), 'm', linewidth = 3.0, label = '5d orbital') # thia takes X axis as index values
            ax13.set(ylabel='Probablilty Density')
            ax13.set(xlabel='R (Angstroms)')
            ax13.set_xlim([0,4])
            #ax13.set_ylim([0,40]) 
            plt.legend(loc=1)
            plt.grid(True)
            ax14 = fig.add_subplot(4,4,14)
            ax14.plot(R, NOCC[13]*(PSTOR[:500, 13]**2), 'k', linewidth = 3.0, label = '4f orbital') # thia takes X axis as index values
            ax14.set(ylabel='Probablilty Density')
            ax14.set(xlabel='R (Angstroms)')
            ax14.set_xlim([0,4])
            #ax14.set_ylim([0,40]) 
            plt.legend(loc=1)
            plt.grid(True)
            ax15 = fig.add_subplot(4,4,15)
            ax15.plot(R, NOCC[14]*(PSTOR[:500, 14]**2), 'b', linewidth = 3.0, label = '6p orbital') # thia takes X axis as index values
            ax15.set(ylabel='Probablilty Density')
            ax15.set(xlabel='R (Angstroms)')
            ax15.set_xlim([0,4])
            #ax15.set_ylim([0,40])
            plt.legend(loc=1)
            plt.grid(True)
            ax16 = fig.add_subplot(4,4,16)
            ax16.plot(R, RHO[:500], 'b', linewidth=3.0, label='Total density of all states') # thia takes X axis as index values
            ax16.set(xlabel='R (Angstroms)')
            ax16.set(ylabel='Probablilty Density')
            ax16.set_xlim([0,4])
            #ax16.set_ylim([0,0.02])                      
            plt.legend(loc=1)
            plt.grid(True)
            plt.show()    
            break
    return 


#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
#      SUBROUTINE HYDRGN(ZSTAR,PSTOR)
# creates hydrogenic wave functions PSTOR with nuclear charge=ZSTAR      
#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

def HYDRGN(ZSTAR,PSTOR):
        
#    store radial parts of hydrogenic wave functions
    for IR in range(NR+1):
        RSTAR = IR*DR*ZSTAR/ABOHR              #scaled radius
        ERSTAR = np.exp(-RSTAR / 2)  # Exponential term for n=1 and n=2 orbitals
        ERSTAR1 = np.exp(-RSTAR / 3)  # Exponential term for n=3 orbitals
        ERSTAR2 = np.exp(-RSTAR / 4)  # Exponential term for n=4 orbitals  
        ERSTAR4 = np.exp(-RSTAR / 4)  # Exponential term for n=4 orbitals
        ERSTAR5 = np.exp(-RSTAR / 5)  # Exponential term for n=5 orbitals
        ERSTAR6 = np.exp(-RSTAR / 6)  # Exponential term for n=6 orbitals
        
        if NOCC[0] != 0:
            PSTOR[IR,0] = RSTAR*ERSTAR**2                #wavefunction for 1s orbital [(Z*r/a)exp(-Z*r/a)]
        if NOCC[1] != 0:
            PSTOR[IR,1] = (2-RSTAR)*RSTAR*ERSTAR         #wavefunction for 2s orbital
        if NOCC[2] != 0:
            PSTOR[IR,2] = (RSTAR**2)*ERSTAR              #wavefunction for 2p orbital
        if NOCC[3] !=0:
            PSTOR[IR, 3] = (1 - (2/3)*RSTAR + ((2/27)*RSTAR**2)) * RSTAR * ERSTAR1        #wavefunction for 3s orbital
        if NOCC[4] !=0:
            PSTOR[IR, 4] = (RSTAR - (RSTAR**2)/6) * RSTAR * ERSTAR1        #wavefunction for 3p orbital
        if NOCC[5] !=0:
            PSTOR[IR, 5] = (1 - RSTAR/4 + (RSTAR**2)/80 - (RSTAR**3)/1920) * RSTAR * ERSTAR2  # Wavefunction for 4s orbital 
        if NOCC[6] != 0:
            PSTOR[IR, 6] = (RSTAR**3) * ERSTAR1  # Wavefunction for 3d orbital 
        if NOCC[7] != 0:
            PSTOR[IR, 7] = (RSTAR - (RSTAR**2)/12 + (RSTAR**3)/240) * RSTAR * ERSTAR2  # Wavefunction for 4p orbital
        if NOCC[8] != 0:  # 5s orbital
            PSTOR[IR, 8] = (1 - RSTAR/5 + (RSTAR**2)/50 - (RSTAR**3)/750 + (RSTAR**4)/12000) * RSTAR * ERSTAR5
        if NOCC[9] != 0:  # 4d orbital
            PSTOR[IR, 9] = (RSTAR**2) * (1 - RSTAR/8 + (RSTAR**2)/192) * ERSTAR4
        if NOCC[10] != 0:  # 5p orbital
            PSTOR[IR, 10] = (RSTAR - (RSTAR**2)/10 + (RSTAR**3)/300) * RSTAR * ERSTAR5
        if NOCC[11] != 0:  # 6s orbital
            PSTOR[IR, 11] = (1 - RSTAR/6 + (RSTAR**2)/72 - (RSTAR**3)/1296 + (RSTAR**4)/31104) * RSTAR * ERSTAR6
        if NOCC[12] != 0:  # 5d orbital
            PSTOR[IR, 12] = (RSTAR**2) * (1 - RSTAR/10 + (RSTAR**2)/300) * ERSTAR5
        if NOCC[13] != 0:  # 4f orbital
            PSTOR[IR, 13] = (RSTAR**3) * ERSTAR4
        if NOCC[14] != 0:  # 6p orbital
            PSTOR[IR, 14] = (RSTAR - (RSTAR**2)/12 + (RSTAR**3)/432) * RSTAR * ERSTAR6

#   normalize wave functions
    for STATE in range(NSTATE):
        if NOCC[STATE] != 0:
            NORM=0
            for IR in range(NR+1):
                NORM = NORM+PSTOR[IR,STATE]**2

            NORM=1/(np.sqrt(NORM*DR))
            for IR in range(NR+1):
                PSTOR[IR,STATE]=PSTOR[IR,STATE]*NORM

    return PSTOR               

#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
#      SUBROUTINE ENERGY(E,FOCK,RHO,PHI,PSTOR)
# subroutine to calculate the energies of a normalized
# set of single-particle wave functions (PSTOR); 
# also calculates Fock terms, density, and electron potential
#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

def ENERGY(E,FOCK,RHO,PHI,PSTOR):
    global Total_Energy_Diff
    
    SOURCE(PSTOR)  #calculate Fock terms and density
    POISSN(RHO)         #calc potntl due to electron charge

    for IE in range(IKEN, ITOT+1):           #zero total energies
        E[NSTATE,IE] = 0


    for STATE in range(NSTATE):
        if NOCC[STATE] != 0:
            for IE in range(IKEN, ITOT+1):     #zero energy for this state
                 E[STATE,IE] = 0
     

            LL1=ANGMOM[STATE]*(ANGMOM[STATE]+1)
            PM=0
            for IR in range(1, NR+1):          #integrate the energy densities
                R = IR*DR
                PZ = PSTOR[IR,STATE]
                PZ2 = PZ**2
                # for sloving singe particle eigen value --> From equation III.30
                E[STATE,IKEN] = E[STATE,IKEN]+(PZ-PM)**2        #kinetic #squared deviation from the mean value of the momentum component (p=-ihbar(del/delx))#This term measures how the momentum of an electron changes as it moves through space(P^2/2m).
                E[STATE,ICEN] = E[STATE,ICEN]+PZ2*LL1/R**2      #centrifugal
                E[STATE,IVEN] = E[STATE,IVEN]-PZ2/R             #elec-nucl
                E[STATE,IVEE] = E[STATE,IVEE]+PHI[IR]*PZ2       #elec-elec
                E[STATE,IVEX] = E[STATE,IVEX]+FOCK[IR,STATE]*PZ #exchange
                PM=PZ     #roll values for derivative

#           put in constant factors
            E[STATE,IKEN] = E[STATE,IKEN]*HBARM/(2*DR)
            E[STATE,ICEN] = E[STATE,ICEN]*DR*HBARM/2       
            E[STATE,IVEN] = E[STATE,IVEN]*ZCHARG*DR
            E[STATE,IVEE] = E[STATE,IVEE]*DR
            E[STATE,IVEX] = E[STATE,IVEX]*DR  
 
#           calculate totals for this state
            E[STATE,IKTOT] = E[STATE,IKEN]+E[STATE,ICEN]
            E[STATE,IVTOT] = E[STATE,IVEN]+E[STATE,IVEE]+E[STATE,IVEX]
            E[STATE,ITOT] = E[STATE,IVTOT]+E[STATE,IKTOT]  #store the single particle energy

#           add this state's contribution to total energy
            for IE in range(IKEN, IVEX+1):
                E[NSTATE,IE]=E[NSTATE,IE]+E[STATE,IE]*NOCC[STATE]
                

    STATE=NSTATE              #calculate total energies
#   don't double count electron-electron and exchange energies
    E[STATE, IVEE] = E[STATE, IVEE]/2
    E[STATE, IVEX] = E[STATE, IVEX]/2
         
#   find total kinetic, potential and total energies
    E[STATE,IKTOT] = E[STATE,IKEN]+E[STATE,ICEN]
    E[STATE,IVTOT] = E[STATE,IVEN]+E[STATE,IVEE]+E[STATE,IVEX]
    E[STATE,ITOT] = E[STATE,IVTOT]+E[STATE,IKTOT]
   
    Total_Energy_Diff = E[NSTATE,ITOT_LAST] - E[NSTATE,ITOT]
    E[NSTATE,ITOT_LAST] = E[NSTATE,ITOT]


    return 

#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
#      SUBROUTINE SOURCE(PSTOR,FOCK,RHO)
# subroutine to compute the density and the Fock terms given PSTOR
#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

def SOURCE(PSTOR):
    for IR in range(0, NR+1):           #include fraction of old density
        RHO[IR] = (1-MIX)*RHO[IR]

   
   #This part calculates the density matrix elements (RHO) using a linear combination of contributions from different states (STATE).      
    for STATE in range(NSTATE):
        if NOCC[STATE] != 0:     #loop over occupied states
            for IR in range(0, NR+1):     #contribution of this state to density
                RHO[IR] = RHO[IR]+MIX*NOCC[STATE]*(PSTOR[IR,STATE]**2)   # Equation III.25 for density
                FOCK[IR,STATE]=0           #zero FOCK term
                
            #This section calculates the Fock matrix (FOCK) elements using integrals involving the radial wave functions (PSTOR). 
            # It involves angular momentum calculations, 3j symbols (SQR3J), and numerical integration loops over the radial coordinate (R).    
#           begin calculation of Fock term
            for STATE2 in range(NSTATE):
                if NOCC[STATE2] != 0:        #loop over occupied states to solve equation III.28b
                    L1 = ANGMOM[STATE]
                    L2 = ANGMOM[STATE2]
                    LSTART = abs(L1-L2)          #limits on ang mom sum
                    LSTOP = L1+L2
                        
                    for LAM in range(int(LSTART), int(LSTOP+1), 2):       #loop over ang mom values, we only need even number because for odd LSTART+LSTOP+LAM, THREEJ is 0.
                        THREEJ = SQR3J(L1,L2,LAM)           #Solution of (l l' lemda)^2 formula III.28b
                        FAC = (-CHARGE/2)*NOCC[STATE2]*THREEJ  # Factor of integral formula III.28b
                        
                        SUM = 0
                        for IR in range(1, NR+1):          #outward integral for Fock
                            R = IR*DR
                            RLAM = R**LAM
                            TERM = PSTOR[IR,STATE2]*PSTOR[IR,STATE]*RLAM/2
                            SUM = SUM+TERM
                            DF = PSTOR[IR,STATE2]*FAC*SUM*DR/(RLAM*R)
                            FOCK[IR,STATE] = FOCK[IR,STATE]+DF
                            SUM = SUM+TERM
                            
                        SUM = 0
                        for IR in range(NR, 0, -1):       #inward integral for Fock
                            R = IR*DR
                            RLAM1 = R**(LAM+1)
                            TERM = PSTOR[IR,STATE2]*PSTOR[IR,STATE]/RLAM1/2
                            SUM = SUM+TERM
                            DF = PSTOR[IR,STATE2]*FAC*SUM*DR*RLAM1/R
                            FOCK[IR,STATE] = FOCK[IR,STATE]+DF
                            SUM=SUM+TERM
                            
    return FOCK, RHO

#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
#      SUBROUTINE POISSN(PHI,RHO)
# subroutine to solve Poisson's equation for the direct potential
# given the electron density RHO
#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

def POISSN(RHO):  # FOR GETTING PHI
    SUM = 0                         #quadrature of density to get 
    for IR in range(1, NR+1):                  # initial value for PHI(1)
        SUM = SUM+RHO[IR]/np.real(IR)    # cumulative sum of these ratios
        
    CON = DR**2/12                   #initial values for outward integ
    SM = 0
    SZ = -CHARGE*RHO[1]/DR
    PHI[0] = 0
    PHI[1] = SUM*CHARGE*DR           

    for IR in range(1, NR):           #Numerov algorithm
        SP = -CHARGE*RHO[IR+1]/((IR+1)*DR)
        PHI[IR+1] = 2*PHI[IR]-PHI[IR-1]+CON*(10*SZ+SP+SM)
        SM = SZ
        SZ = SP

    M = (PHI[NR]-PHI[NR-10])/(10*DR)   #subtract off linear behavior

    for IR in range(1, NR+1): 
        R = IR*DR
        PHI[IR] = PHI[IR]/R-M       #factor of 1/r for true potl

    return PHI

#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
#      SUBROUTINE SNGWFN(STATE,E,PSTOR,FOCK,PHI)
# subroutine to solve the single-particle wave function as an
# inhomogeneous boundary-value problem for a given state, energy E,
# source term (FOCK), and potential (PHI)
#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

def SNGWFN(STATE,E,PSTOR,FOCK,PHI):
    DRHBM = DR**2/HBARM/6           #useful constants
    LL1 = ANGMOM[STATE]*(ANGMOM[STATE]+1)*HBARM/2
    
    K2M = 0                         #integrate outward homogeneous soln
    K2Z = DRHBM*(E-PHI[1]+(ZCHARG-LL1/DR)/DR)  # constatnts of Formula III.28a with F = 0
    PSIOUT[0] = 0
    PSIOUT[1] = 1E-10
    for IR in range(2, NR+1):
        R = DR*IR                 #Numerov algorithm
        K2P = DRHBM*(E-PHI[IR]+(ZCHARG-LL1/R)/R)
        PSIOUT[IR] = (PSIOUT[IR-1]*(2-10*K2Z)-PSIOUT[IR-2]*(1+K2M))/(1+K2P)
        K2M = K2Z                 #roll values
        K2Z = K2P

    K2P = 0                         #integrate inward homogeneous soln
    R = (NR-1)*DR
    K2Z = DRHBM*(E-PHI[NR-1]+(ZCHARG-LL1/R)/R)
    PSIIN[NR] = 0
    PSIIN[NR-1] = 1E-10
    for IR in range(NR-2,0,-1):
        R = DR*IR                 #Numerov algorithm
        K2M = DRHBM*(E-PHI[IR]+(ZCHARG-LL1/R)/R)
        PSIIN[IR]=(PSIIN[IR+1]*(2-10*K2Z)-PSIIN[IR+2]*(1+K2P))/(1+K2M)
        K2P = K2Z                 #roll values
        K2Z = K2M


#The code calculates the Wronskian at the middle of the mesh, which is a quantity used in quantum mechanics 
# to ensure consistency between the inward and outward solutions.
#
    NR2 = NR//2                       #Wronskian at middle of mesh
    
    WRON = (PSIIN[NR2+1]-PSIIN[NR2-1])/(2*DR)*PSIOUT[NR2]
    WRON = WRON-(PSIOUT[NR2+1]-PSIOUT[NR2-1])/(2*DR)*PSIIN[NR2]
 
    SUM = 0                           #outward integral in Green's soln
    for IR in range(1, NR+1):
        TERM = -PSIOUT[IR]*FOCK[IR,STATE]/2
        SUM = SUM+TERM
        PSTOR[IR,STATE] = PSIIN[IR]*SUM*DR
        SUM = SUM+TERM
        
                
    SUM = 0                           #inward integral in Green's soln
    for IR in range(NR,0,-1):
        TERM = -PSIIN[IR]*FOCK[IR,STATE]/2
        SUM = SUM+TERM
        PSTOR[IR,STATE] = (PSTOR[IR,STATE]+PSIOUT[IR]*SUM*DR)/WRON
        SUM = SUM+TERM

#specific block of code that keeps the 1s and 2s states orthogonal by subtracting their overlap.

    if STATE == 1:         #keep 1s and 2s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,0]*PSTOR[IR,1]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,1] = PSTOR[IR,1]-SUM*PSTOR[IR,0]
            
    if STATE == 3:         #keep 1s and 3s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,0]*PSTOR[IR,3]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,3] = PSTOR[IR,3]-SUM*PSTOR[IR,0]
    

    if STATE == 3:         #keep 2s and 3s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,1]*PSTOR[IR,3]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,3] = PSTOR[IR,3]-SUM*PSTOR[IR,1]
            
    if STATE == 4:         #keep 2p and 3p states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,2]*PSTOR[IR,4]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,4] = PSTOR[IR,4]-SUM*PSTOR[IR,2]
    
    if STATE == 5:         #keep 1s and 4s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,0]*PSTOR[IR,5]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,5] = PSTOR[IR,5]-SUM*PSTOR[IR,0]
            
    if STATE == 5:         #keep 2s and 4s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,1]*PSTOR[IR,5]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,5] = PSTOR[IR,5]-SUM*PSTOR[IR,1]
    
    if STATE == 5:         #keep 3s and 4s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,3]*PSTOR[IR,5]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,5] = PSTOR[IR,5]-SUM*PSTOR[IR,3]
            
    if STATE == 7:         #keep 2p and 4p states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,2]*PSTOR[IR,7]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,7] = PSTOR[IR,7]-SUM*PSTOR[IR,2]
            
    if STATE == 7:         #keep 3p and 4p states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,4]*PSTOR[IR,7]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,7] = PSTOR[IR,7]-SUM*PSTOR[IR,4]
            
    if STATE == 8:         #keep 1s and 5s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,0]*PSTOR[IR,8]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,8] = PSTOR[IR,8]-SUM*PSTOR[IR,0]
            
    if STATE == 8:         #keep 2s and 5s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,1]*PSTOR[IR,8]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,8] = PSTOR[IR,8]-SUM*PSTOR[IR,1]
            
    if STATE == 8:         #keep 3s and 5s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,3]*PSTOR[IR,8]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,8] = PSTOR[IR,8]-SUM*PSTOR[IR,3]
            
    if STATE == 8:         #keep 4s and 5s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,5]*PSTOR[IR,8]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,8] = PSTOR[IR,8]-SUM*PSTOR[IR,5]
            
    if STATE == 9:         #keep 3d and 4d states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,6]*PSTOR[IR,9]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,9] = PSTOR[IR,9]-SUM*PSTOR[IR,6]
            
    if STATE == 10:         #keep 2p and 5p states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,2]*PSTOR[IR,10]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,10] = PSTOR[IR,10]-SUM*PSTOR[IR,2]
            
    if STATE == 10:         #keep 3p and 5p states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,4]*PSTOR[IR,10]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,10] = PSTOR[IR,10]-SUM*PSTOR[IR,4]
            
    if STATE == 10:         #keep 4p and 5p states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,7]*PSTOR[IR,10]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,10] = PSTOR[IR,10]-SUM*PSTOR[IR,7]
            
    if STATE == 11:         #keep 1s and 6s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,0]*PSTOR[IR,11]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,11] = PSTOR[IR,11]-SUM*PSTOR[IR,0]
            
    if STATE == 11:         #keep 2s and 6s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,1]*PSTOR[IR,11]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,11] = PSTOR[IR,11]-SUM*PSTOR[IR,1]
            
    if STATE == 11:         #keep 3s and 6s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,3]*PSTOR[IR,11]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,11] = PSTOR[IR,11]-SUM*PSTOR[IR,3]
            
    if STATE == 11:         #keep 4s and 6s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,5]*PSTOR[IR,11]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,11] = PSTOR[IR,11]-SUM*PSTOR[IR,5]
            
    if STATE == 11:         #keep 5s and 6s states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,8]*PSTOR[IR,11]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,11] = PSTOR[IR,11]-SUM*PSTOR[IR,8]
            
    if STATE == 12:         #keep 3d and 5d states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,6]*PSTOR[IR,12]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,12] = PSTOR[IR,12]-SUM*PSTOR[IR,6]
            
    if STATE == 12:         #keep 4d and 5d states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,9]*PSTOR[IR,12]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,12] = PSTOR[IR,12]-SUM*PSTOR[IR,9]
            
    if STATE == 14:         #keep 2p and 6p states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,2]*PSTOR[IR,14]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,14] = PSTOR[IR,14]-SUM*PSTOR[IR,2]
            
    if STATE == 14:         #keep 3p and 6p states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,4]*PSTOR[IR,14]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,14] = PSTOR[IR,14]-SUM*PSTOR[IR,4]
            
    if STATE == 14:         #keep 4p and 6p states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,7]*PSTOR[IR,14]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,14] = PSTOR[IR,14]-SUM*PSTOR[IR,7]
            
    if STATE == 14:         #keep 5p and 6p states orthogonal
        SUM = 0
        for IR in range(0, NR+1):
            SUM = SUM+PSTOR[IR,10]*PSTOR[IR,14]
           
        SUM = SUM*DR
        for IR in range(0, NR+1):
            PSTOR[IR,14] = PSTOR[IR,14]-SUM*PSTOR[IR,10]
            
    NORM = 0                          #normalize the soln
    for IR in range(0, NR+1):
        NORM = NORM+PSTOR[IR,STATE]**2
        
    NORM = 1/np.sqrt(NORM*DR)
    for IR in range(0,NR+1):
        PSTOR[IR,STATE] = PSTOR[IR,STATE]*NORM

    return PSTOR

#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
#      SUBROUTINE SQR3J(L1,L2,LAM,THREEJ)
# subroutine to calculate square of the 3-j coefficient appearing
#                in the exchange energy
#CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

def SQR3J(L1,L2,LAM):
    IMAX = L1+L2+LAM+1           #useful combinations
    P = (L1+L2+LAM)/2
    DELTA = FACTRL[int(L1 + L2 - LAM)] * FACTRL[int(-L1 + L2 + LAM)]
    DELTA = DELTA * FACTRL[int(L1 - L2 + LAM)] / FACTRL[int(IMAX)]
    THREEJ = DELTA * (FACTRL[int(P)]**2)
    THREEJ = THREEJ / (FACTRL[int(P - L1)]**2)
    THREEJ = THREEJ / (FACTRL[int(P - L2)]**2)
    THREEJ = THREEJ / (FACTRL[int(P - LAM)]**2)
    
    return THREEJ


## 주기율표 1~3주기 ##

data = {
    'Atomic number': ['1', '2',
                      '3', '4', '5', '6', '7', '8', '9', '10', 
                      '11', '12', '13', '14', '15', '16', '17', '18', 
                      '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', 
                      '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
                      '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86'],
    'Element symbol': ['H-', 'He', 
                       'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 
                       'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 
                       'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr',
                       'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe',
                       'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn'],
    'Period': [1, 1, 
               2, 2, 2, 2, 2, 2, 2, 2, 
               3, 3, 3, 3, 3, 3, 3, 3, 
               4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 
               5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
               6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
    'Orbital occupation(1s, 2s, 2p, 3s, 3p, 4s, 3d, 4p, 5s, 4d, 5p, 6s, 5d, 4f, 6p)': [(2,0,0,0,0,0,0,0,0,0,0,0,0,0,0), (2,0,0,0,0,0,0,0,0,0,0,0,0,0,0), 
                                                (2,1,0,0,0,0,0,0,0,0,0,0,0,0,0), (2,2,0,0,0,0,0,0,0,0,0,0,0,0,0), (2,2,1,0,0,0,0,0,0,0,0,0,0,0,0), (2,2,2,0,0,0,0,0,0,0,0,0,0,0,0), (2,2,3,0,0,0,0,0,0,0,0,0,0,0,0), (2,2,4,0,0,0,0,0,0,0,0,0,0,0,0), (2,2,5,0,0,0,0,0,0,0,0,0,0,0,0), (2,2,6,0,0,0,0,0,0,0,0,0,0,0,0), 
                                                (2,2,6,1,0,0,0,0,0,0,0,0,0,0,0), (2,2,6,2,0,0,0,0,0,0,0,0,0,0,0), (2,2,6,2,1,0,0,0,0,0,0,0,0,0,0), (2,2,6,2,2,0,0,0,0,0,0,0,0,0,0), (2,2,6,2,3,0,0,0,0,0,0,0,0,0,0), (2,2,6,2,4,0,0,0,0,0,0,0,0,0,0), (2,2,6,2,5,0,0,0,0,0,0,0,0,0,0), (2,2,6,2,6,0,0,0,0,0,0,0,0,0,0), 
                                                (2,2,6,2,6,1,0,0,0,0,0,0,0,0,0), (2,2,6,2,6,2,0,0,0,0,0,0,0,0,0), (2,2,6,2,6,2,1,0,0,0,0,0,0,0,0), (2,2,6,2,6,2,2,0,0,0,0,0,0,0,0), (2,2,6,2,6,2,3,0,0,0,0,0,0,0,0), (2,2,6,2,6,1,5,0,0,0,0,0,0,0,0), (2,2,6,2,6,2,5,0,0,0,0,0,0,0,0), (2,2,6,2,6,2,6,0,0,0,0,0,0,0,0), 
                                                (2,2,6,2,6,2,7,0,0,0,0,0,0,0,0), (2,2,6,2,6,2,8,0,0,0,0,0,0,0,0), (2,2,6,2,6,1,10,0,0,0,0,0,0,0,0), (2,2,6,2,6,2,10,0,0,0,0,0,0,0,0), (2,2,6,2,6,2,10,1,0,0,0,0,0,0,0), (2,2,6,2,6,2,10,2,0,0,0,0,0,0,0), (2,2,6,2,6,2,10,3,0,0,0,0,0,0,0), (2,2,6,2,6,2,10,4,0,0,0,0,0,0,0), 
                                                (2,2,6,2,6,2,10,5,0,0,0,0,0,0,0), (2,2,6,2,6,2,10,6,0,0,0,0,0,0,0), (2,2,6,2,6,2,10,6,1,0,0,0,0,0,0), (2,2,6,2,6,2,10,6,2,0,0,0,0,0,0), (2,2,6,2,6,2,10,6,2,1,0,0,0,0,0), (2,2,6,2,6,2,10,6,2,2,0,0,0,0,0), (2,2,6,2,6,2,10,6,1,4,0,0,0,0,0), (2,2,6,2,6,2,10,6,1,5,0,0,0,0,0),
                                                (2,2,6,2,6,2,10,6,2,5,0,0,0,0,0), (2,2,6,2,6,2,10,6,1,7,0,0,0,0,0), (2,2,6,2,6,2,10,6,1,8,0,0,0,0,0), (2,2,6,2,6,2,10,6,0,10,0,0,0,0,0), (2,2,6,2,6,2,10,6,1,10,0,0,0,0,0), (2,2,6,2,6,2,10,6,2,10,0,0,0,0,0), (2,2,6,2,6,2,10,6,2,10,1,0,0,0,0), (2,2,6,2,6,2,10,6,2,10,2,0,0,0,0),
                                                (2,2,6,2,6,2,10,6,2,10,3,0,0,0,0), (2,2,6,2,6,2,10,6,2,10,4,0,0,0,0), (2,2,6,2,6,2,10,6,2,10,5,0,0,0,0), (2,2,6,2,6,2,10,6,2,10,6,0,0,0,0), (2,2,6,2,6,2,10,6,2,10,6,1,0,0,0), (2,2,6,2,6,2,10,6,2,10,6,2,0,0,0), (2,2,6,2,6,2,10,6,2,10,6,2,1,0,0), (2,2,6,2,6,2,10,6,2,10,6,2,1,1,0),
                                                (2,2,6,2,6,2,10,6,2,10,6,2,0,3,0), (2,2,6,2,6,2,10,6,2,10,6,2,0,4,0), (2,2,6,2,6,2,10,6,2,10,6,2,0,5,0), (2,2,6,2,6,2,10,6,2,10,6,2,0,6,0), (2,2,6,2,6,2,10,6,2,10,6,2,0,7,0), (2,2,6,2,6,2,10,6,2,10,6,2,1,7,0), (2,2,6,2,6,2,10,6,2,10,6,2,0,9,0), (2,2,6,2,6,2,10,6,2,10,6,2,0,10,0),
                                                (2,2,6,2,6,2,10,6,2,10,6,2,0,11,0), (2,2,6,2,6,2,10,6,2,10,6,2,0,12,0), (2,2,6,2,6,2,10,6,2,10,6,2,0,13,0), (2,2,6,2,6,2,10,6,2,10,6,2,0,14,0), (2,2,6,2,6,2,10,6,2,10,6,2,1,14,0), (2,2,6,2,6,2,10,6,2,10,6,2,2,14,0), (2,2,6,2,6,2,10,6,2,10,6,2,3,14,0), (2,2,6,2,6,2,10,6,2,10,6,2,4,14,0),
                                                (2,2,6,2,6,2,10,6,2,10,6,2,5,14,0), (2,2,6,2,6,2,10,6,2,10,6,2,6,14,0), (2,2,6,2,6,2,10,6,2,10,6,2,7,14,0), (2,2,6,2,6,2,10,6,2,10,6,1,9,14,0), (2,2,6,2,6,2,10,6,2,10,6,1,10,14,0), (2,2,6,2,6,2,10,6,2,10,6,2,10,14,0), (2,2,6,2,6,2,10,6,2,10,6,2,10,14,1), (2,2,6,2,6,2,10,6,2,10,6,2,10,14,2),
                                                (2,2,6,2,6,2,10,6,2,10,6,2,10,14,3), (2,2,6,2,6,2,10,6,2,10,6,2,10,14,4), (2,2,6,2,6,2,10,6,2,10,6,2,10,14,5), (2,2,6,2,6,2,10,6,2,10,6,2,10,14,6)]
}
'''

data = {
    'Atomic number': ['1','2','3','4','5','6','7','8','9','10'],
    'Element symbol': ['H-', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne'],
    'Period': [1, 1, 2, 2, 2, 2, 2, 2, 2, 2],
    'Orbital occupation(1s, 2s, 2p)': [(2,0,0,0,0), (2,0,0,0,0), 
                                                (2,1,0,0,0), (2,2,0,0,0), (2,2,1,0,0), 
                                                (2,2,2,0,0), (2,2,3,0,0), (2,2,4,0,0), 
                                                (2,2,5,0,0), (2,2,6,0,0)]
}
'''

periodic_table = pd.DataFrame(data)

def display_element_info(period, element):
    selected_row = periodic_table[(periodic_table['Period'] == period) & (periodic_table['Element symbol'] == element)]
    z = int(selected_row['Atomic number'].values[0])
    orbital_info = selected_row['Orbital occupation(1s, 2s, 2p, 3s, 3p, 4s, 3d, 4p, 5s, 4d, 5p, 6s, 5d, 4f, 6p)'].values[0]
    orbital_1s = orbital_info[0]
    orbital_2s = orbital_info[1]
    orbital_2p = orbital_info[2]
    orbital_3s = orbital_info[3]
    orbital_3p = orbital_info[4]
    orbital_4s = orbital_info[5]
    orbital_3d = orbital_info[6]
    orbital_4p = orbital_info[7]
    orbital_5s = orbital_info[8]
    orbital_4d = orbital_info[9]
    orbital_5p = orbital_info[10]
    orbital_6s = orbital_info[11]
    orbital_5d = orbital_info[12]
    orbital_4f = orbital_info[13]
    orbital_6p = orbital_info[14]


## 주기별 원소 표 ##
    filtered_table = periodic_table[(periodic_table['Period'] == period) & (periodic_table['Element symbol'] == element)]
    #filtered_table = periodic_table[periodic_table['Period'] == period]  
    styled_table = filtered_table.style.set_table_styles([
        {'selector': 'td', 'props': [('padding', '5px'), ('text-align', 'center'), ('width', '50px')]},
        {'selector': 'th', 'props': [('padding', '5px'), ('text-align', 'center'), ('width', '50px')]},
    ]).hide_index()  # 인덱스 숨기기
    styled_table.set_table_attributes('style="width:100%; table-layout: fixed;"')
    display(HTML(styled_table.render()))


    if period == 1 and element == 'H-':
        z = 1  # 원소 H의 원자 번호는 1
        orbital_1s = 2  # 원소 H의 1s 오비탈 전자 수는 2

    INIT()
    PARAM(z, orbital_1s, orbital_2s, orbital_2p, orbital_3s, orbital_3p, orbital_4s, orbital_3d, orbital_4p, orbital_5s, orbital_4d, orbital_5p, orbital_6s, orbital_5d, orbital_4f, orbital_6p)
    ARCHON()

    # print(f"Atomic number: {z}")
    # print(f"Orbital occupation: 1s={orbital_1s}, 2s={orbital_2s}, 2p={orbital_2p}, 3s={orbital_3s}, 3p={orbital_3p}")

# The functions below require a virtual function implementation that replaces the actual contents of ARCHON, INIT, PARAM, etc.

period_dropdown = Dropdown(options={'Period 1': 1, 'Period 2': 2, 'Period 3': 3, 'Period 4': 4, 'Period 5': 5, 'Period 6': 6}, description='Selected Period:',
                           style={'description_width': 'initial'}, layout={'width': '200px'})

element_dropdown = lambda period: Dropdown(options=periodic_table[periodic_table['Period'] == period]['Element symbol'].unique().tolist(),
                                           description='Selected Element:', style={'description_width': 'initial'}, layout={'width': '200px'})

ui = interactive(display_element_info, period=period_dropdown, element=element_dropdown(period_dropdown.value))
ui.children[1].layout = Layout(width='200px')

def update_element_options(*args):
    period = period_dropdown.value
    ui.children[1].options = periodic_table[periodic_table['Period'] == period]['Element symbol'].unique().tolist()

period_dropdown.observe(update_element_options, 'value')

display(HBox([ui.children[0], ui.children[1]]))
display(ui.children[-1])



Output()

### "Hartree-Fock Solutions of Small Atomic Systems (Central-Field Approximation)"
##### Reference: COMPUTATIONAL PHYSICS (Fortran Version) by Steven E. Koonin & Dawn C. Meredith
#### Dr. Kaptan Rajput, (Prof. Yong-Hoon Kim group) School of Electrical Engineering, Korea Advanced Institute of Science and Technology (KAIST)

###### Last updated : 2024. 05. 14

###### To solve:
 $ -\dfrac{\hbar^2}{2m} \, \dfrac{\mathrm{d}^2 \psi}{\mathrm{d} r^2} + V_{\text{eff}}(r)\psi = H\psi = E\psi $ 
 
###### Numerical implementation:

1. **Computation of Initial Guessed Optimal Hydrogenic Wavefunction:**

   - The wavefunctions for orbitals are described as follows:
   
 $  Ψ_{1s}(r) = 2\left(\frac{Z^*}{a}\right)^{\frac{1}{2}} \cdot \frac{Z^*r}{a}e^{-\frac{Z^*r}{a}} $ 
 
 $  Ψ_{2s}(r) = 2\left(\frac{Z^*}{2a}\right)^{\frac{1}{2}} \cdot \left(1-\frac{Z^*r}{2a}\right) \cdot \frac{Z^*r}{2a}e^{-\frac{Z^*r}{2a}} $

 $  Ψ_{2p}(r) = 2\left(\frac{Z^*}{3a}\right)^{\frac{1}{2}} \left(\frac{Z^*r}{2a}\right)^2 e^{-\frac{Z^*r}{2a}} $
   
   where Z* and 𝑎 represent the effective charge and Bohr radius. 

   - To initiate the iterative procedure, normalized hydrogenic wavefunctions Ψ (1s, 2s, & 2p) and their energies are obtained according to the selected (input from user) period and element.
 
   - The variational parameter Z* computed from the virial theorem.
   
   - This Z* used to calculated the initial optimal wavefunction for a hydrogenic system.

   - Further information of several other wavefunctions can be obtained from page no. 359 of Quantum Mechanics: Concepts and Applications" book by Nouredine Zettili.
   

2. **Evaluation of Single Particle Energies and Total Energy:**
   - Utilizing the wavefunction obtained in the initial step, single particle energies and the total energy of the system are calculated.
   

3. **Iterative Energy Convergence:**
   - The total energy is compared against a specified tolerance threshold.
   - If the convergence criterion is not met, the total electron density is updated using a weighted average of the current and previous densities.
   - This iterative process continues until the total energy converges within the specified tolerance.
   

4. **Printout of Energy Components:**
   - The following energy components are printed for each occupied state:
     - Kinetic energy (Ktot)
     - Electron-nucleus potential energy (Vev)
     - Electron-electron potential energy (Vee)
     - Exchange energy (Vex)
     - Total potential energy (Vtot)
     - Total energy (Etot)
   - Additionally, the total energy (KTOT, VENTOT, VEETOT, VEXTOT, VTOT, Total energy) for all states is also printed.
   - Visualization of orbital probability and the total probability densities as a function of radius is facilitated.

Note: This program is designed to solve the Hartree-Fock equations within the filling approximation for atomic systems containing electrons distributed among the 1s, 2s, 2p, 3s, 3p, 4s, 3d, 4p, 5s, 4d, 5p, 6s, 5d, 4f, and 6p atomic orbitals. It is capable of computing properties for atoms up to Radon (Rn). For calculations involving atoms beyond Radon, additional orbital shell information would need to be provided in the code.